# Text Mining Group Project
## CS-EN Corpus

##### TOC for Implemented Metrics.

---

In [1]:
# Necessary Installs.
#!pip install rouge

In [2]:
# Imports
import pandas as pd
import numpy as np
from collections import Counter
from rouge import Rouge
import string
from nltk.translate import chrf_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import StandardScaler

Load Dataset

In [3]:
df1 = pd.read_csv("scores.csv")

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11585 entries, 0 to 11584
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       11585 non-null  object 
 1   reference    11585 non-null  object 
 2   translation  11585 non-null  object 
 3   z-score      11585 non-null  float64
 4   avg-score    11585 non-null  float64
 5   annotators   11585 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 543.2+ KB


In [5]:
df1.head()

,source,reference,translation,z-score,avg-score,annotators
0,Uchopíte pak zbraň mezi své předloktí a rameno...,You will then grab the weapon between your for...,You then grasp the gun between your forearm an...,-0.675383,60.000000,3
1,"Ale je-li New York změna, pak je to také znovu...","But if New York is changed, then it's also a r...","But if New York is change, it is also reinvent...",-0.829403,44.000000,2
2,"Dlouho a intenzivně jsem během léta přemýšlel,...",I have been thinking over and over again over ...,I have thought long and hard over the course o...,0.803185,96.500000,2
3,"Najdou si jiný způsob, jak někde podvádět.",They find another way to cheat somewhere.,They will find another way how to defraud others.,0.563149,90.500000,2
4,Zpráva o výměně v čele prezidentovy administra...,The report on the replacement of the president...,The news of the replacement at the top of the ...,0.021549,74.666667,3


---
### PreProcessing

In [6]:
# Check for empty or sparse reference / translation, and drop them.
for column in ["source","reference","translation"]:
    print(column)
    bad_idx = [idx for idx in np.where(df1[column].str.len()<=2)[0]]
    if bad_idx != []:
        print(df1.iloc[bad_idx])
    print(f"Bad idx: {bad_idx}")
    df1 = df1.drop(index=bad_idx)

source
Bad idx: []
reference
Bad idx: []
translation
Bad idx: []


In [7]:
# Create two dataframes. One is unprocessed, the other is preprocessed to remove punctuation and be lowercased.
# (Future note: maybe also remove stopwords?)
df_u = df1.copy()
df_p = df1.copy()

for x in ["source","reference","translation"]:
    # lowercase.
    df_p[x] = df1[x].str.lower()
    # Remove punct.
    df_p[x] = df1[x].map(lambda s: s.translate(str.maketrans("","",string.punctuation))
                           .lower()
                          )
    
df_dict = {"df_u":df_u,"df_p":df_p}

In [8]:
## Initialize a scaler for later.
scaler = StandardScaler()

---
### Metrics

--- 
Bleu Metric

In [9]:
def my_first_BLEU(reference,translation):
    """
    Expects lists of strings for both reference and translation.
    Returns the score 
    """
    
    # Let word be every unique word in the translation.
    # Can be done by setting up a Counter object.
    t_c = Counter(word_tokenize(translation))
    words = sorted(t_c)
    
    refs_c = Counter(word_tokenize(reference))
    
    # Let Covered be the minimum amt of times a word appears in the reference, compared to R(w).
    # Let D(word) be how many times a unique word appears in the candidate translation.
    # Let R(word) be the largest numer of times the word appears in any one reference.
    
    covered = 0
    
    for word in words:
        covered += min(t_c[word],refs_c[word])

    

    # Let total be the number of words in translation.
    total = sum(t_c.values())
    
    BLEU_score = covered / total
    
    return BLEU_score


In [10]:
bleu_scores_list = ["basic bleu"]

for df in list(df_dict.values()):
    for key in bleu_scores_list:
        # Apply the function to get a column of the scores.
        df[(key+" score")] = df.apply(lambda row: my_first_BLEU(row["reference"],row["translation"]),axis=1)
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
Rouge metric as described in

https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460

And
https://pypi.org/project/rouge-metric/

In [11]:
rouge = Rouge()

In [12]:
# Example Cell
model_out = df1["translation"][0]
reference = df1["reference"][0]
# The get scores method returns three metrics, F1 score, p precision and recall r.
# For each unigram,bigram and Longest sequence.
rouge.get_scores(model_out,reference)

[{'rouge-1': {'f': 0.6976744136289887, 'p': 0.75, 'r': 0.6521739130434783},
  'rouge-2': {'f': 0.2926829218560381,
   'p': 0.3157894736842105,
   'r': 0.2727272727272727},
  'rouge-l': {'f': 0.6999999950500001,
   'p': 0.7777777777777778,
   'r': 0.6363636363636364}}]

In [13]:
# Get scores.
# For every df considered;
for df in list(df_dict.values()):
    # For the entire model, model_out and reference need to be lists of strings.
    model_out = df["translation"].to_list()
    reference = df["reference"].to_list()
    rouge_scores = rouge.get_scores(model_out,reference)
    # For each of the scores calculated, output a new column in the df with the f1 scores.
    for key in rouge_scores[0].keys():
        df[(key+" score")] = pd.Series([score[key]["f"] for score in rouge_scores])
        # Also add a z score column.
        df[(key+ "_zscore")] = scaler.fit_transform(df[(key+" score")].to_numpy().reshape(-1,1)).flatten()

---
#### chrF metric

Check the paper here: https://www.aclweb.org/anthology/W15-3049.pdf

The general formula for the CHRF score is:

`CHRFBeta = (1 + Beta**2) * ((chrP * chrR) / (Beta**2*chrP + chrR))`

where:
* chrP is the percentage of n-grams in the hypothesis which have a counterpart in the reference.
* chrR is the percentage of character n-grams in the reference which are also present in the hypothesis.
* Beta is a parameter which assigns beta times more importance to recall than to precision (if beta == 1, they have the same importance).

In [14]:
# I was surprised, but this works exactly like it's intended. Makes a new column with the chrF score for each row of the df.
# The default n-gram values are min == 1, max == 6. 
# The default beta is 3.

# Moreover, it is worthwhile to mention chrf uses its own tokenization with whitespaces.
# All parameters to test chrf scores with. feel free to play around with this and test out different combinations.
# Note: this takes a few minutes to run.
min_len = [1,2]
max_len = [6,10]
beta = [1,3]

for df in list(df_dict.values()):
    chrf_scores=[]
    for min_l in min_len:
        for max_l in max_len:
            for b in beta:
                append_str = "chrf_b" + str(b) + "_n" + str(min_l) + str(max_l)
                chrf_scores.append(append_str)
                df[append_str] = df.apply(lambda row: chrf_score.sentence_chrf(row["reference"],row["translation"],min_len=min_l,max_len=max_l,beta=b),axis=1)
                # Also add a z score column.
                df[(append_str+ "_zscore")] = scaler.fit_transform(df[append_str].to_numpy().reshape(-1,1)).flatten()

df_p.loc[:,chrf_scores]

,chrf_b1_n16,chrf_b3_n16,chrf_b1_n110,chrf_b3_n110,chrf_b1_n26,chrf_b3_n26,chrf_b1_n210,chrf_b3_n210
0,0.606163,0.593047,0.481158,0.470592,0.550098,0.538111,0.436121,0.426466
1,0.570696,0.543095,0.489202,0.464494,0.516160,0.490795,0.449850,0.426704
2,0.546121,0.583650,0.428934,0.458783,0.486924,0.520605,0.383026,0.409883
3,0.450730,0.483875,0.358125,0.385747,0.378714,0.407239,0.307826,0.332269
4,0.421882,0.402661,0.304163,0.290193,0.341553,0.325889,0.246456,0.235046
...,...,...,...,...,...,...,...,...
11580,0.539258,0.551747,0.446251,0.456770,0.485784,0.497125,0.406209,0.415871
11581,0.506220,0.493031,0.363147,0.353604,0.429060,0.417804,0.304384,0.296320
11582,0.814027,0.833338,0.769553,0.788159,0.795701,0.814677,0.754429,0.772772
11583,0.590038,0.548472,0.497672,0.462308,0.534946,0.497122,0.456803,0.424207


---
### Comparison of Applied Metrics
Because the numeric system used for all of these can be different, the best way to compare them is by checking the correlation with the annotator's scores.

In [15]:
# Initialize a dict to be transformed to a df later, for score comparison.
corr_list = ["pearson","kendall"]
scores_dict = {}

for df_name in df_dict.keys():
    for corr in corr_list:
        scores_dict[corr+"_"+df_name] = []
    
scores_index = []

In [16]:
# Thankfully, Pandas has a corr method. Use it on standardized scores obtained previously.

# for each declared corr method, compute the corr between each computed metric and the avg-score column for each considered df.
for df_name in df_dict.keys():
    for corr in corr_list:
        for key in rouge_scores[0].keys():
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for chrf_score in set(chrf_scores):
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(chrf_score+"_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))
        for bleu_score in bleu_scores_list:
            scores_dict[corr+"_"+df_name].append(df_dict[df_name].loc[:,(key+ "_zscore")].corr(df_dict[df_name].loc[:,"z-score"],method=corr))


# Build also a list that will be used to create the index for the scores dataframe.
scores_index.extend(list(rouge_scores[0].keys()))
scores_index.extend(chrf_scores)
scores_index.extend(bleu_scores_list)

In [17]:
scores_df = pd.DataFrame(scores_dict,index=scores_index)
scores_df

,pearson_df_u,kendall_df_u,pearson_df_p,kendall_df_p
rouge-1,0.415136,0.279233,0.426726,0.287188
rouge-2,0.377920,0.260021,0.390356,0.267990
rouge-l,0.422805,0.284842,0.437133,0.294956
chrf_b1_n16,0.431080,0.293057,0.426666,0.289482
chrf_b3_n16,0.412850,0.276835,0.403465,0.270349
chrf_b1_n110,0.446626,0.302963,0.441723,0.299204
chrf_b3_n110,0.417907,0.281547,0.409741,0.275908
chrf_b1_n26,0.404970,0.275193,0.398291,0.270509
chrf_b3_n26,0.426657,0.291160,0.422792,0.287835
chrf_b1_n210,0.404136,0.273356,0.396626,0.268242


In [18]:
for column in scores_df.columns:
    print(f"Max in {column}:")
    print("{}, with {}".format(scores_df[column].argmax(),scores_df[column].max()))


Max in pearson_df_u:
10, with 0.4507125571904485
Max in kendall_df_u:
10, with 0.30454719913977363
Max in pearson_df_p:
10, with 0.4451480911788279
Max in kendall_df_p:
10, with 0.3004565161769697
